<a href="https://colab.research.google.com/github/MaundoJako/Data-Engineering-Bootcamp/blob/main/Week-3_Workshop/Homework_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)

sum_of_outputs = sum(generator)

print("Sum of outputs for limit =", limit, ":", sum_of_outputs)


Sum of outputs for limit = 5 : 8.382332347441762


In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 13
generator = square_root_generator(limit)

for _ in range(12):
    next(generator)

thirteenth_value = next(generator)
print("13th number yielded for limit =", limit, ":", thirteenth_value)


13th number yielded for limit = 13 : 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [11]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# ANSWER:

In [45]:
import duckdb

# Creating a connection to DuckDB:
conn = duckdb.connect(database=':memory:', read_only=False)

# Defining a function to load data into DuckDB:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Loading the first generator:
load_data(people_1(), "people_table1")

# Loading the second generator (into a seperate table):
load_data(people_2(), "people_table2")

# Merging both tables into a new table:
conn.execute("CREATE TABLE merged_table AS SELECT * FROM people_table1 UNION ALL SELECT * FROM people_table2")

# Calculating the sum of ages for all people from the table:
result = conn.execute("SELECT SUM(Age) FROM merged_table").fetchall()
sum_of_ages = result[0][0]

print("Sum of ages for all people:", sum_of_ages)


Sum of ages for all people: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [23]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [46]:
import dlt
import duckdb

# Re-use the generators from Exercise 2
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Define the connection to load to using dlt
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')

# Load the first generator with primary key ID
pipeline.run(people_1(), table_name="people_table1", write_disposition="replace", primary_key='ID')

# Load the second generator with merge, replacing records with overlapping IDs
pipeline.run(people_2(), table_name="people_table1", write_disposition="merge", primary_key='ID')

# After the pipeline execution, use duckdb connection to fetch the result
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
result = conn.execute("SELECT SUM(Age) FROM people_data_staging.people_table1").fetchall()
sum_of_ages = result[0][0]

print("Sum of ages for all people:", sum_of_ages)




Sum of ages for all people: 213


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX